In [1]:
import os
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
import pandas as pd

import torch
from Models import *
from functions import *

from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split

In [2]:
x = np.genfromtxt('./generated_data/pivot_traffic.csv', delimiter=',', skip_header=1, usecols=range(1, 17), dtype=float)
y = np.genfromtxt('./generated_data/pivot_PM10.csv', delimiter=',', skip_header=1, usecols=(1,2),  dtype=float)

In [3]:
print(x.shape)
print(y.shape)
print(x[:,0].shape)
print(y[:,0].shape)

(6510, 16)
(6510, 2)
(6510,)
(6510,)


In [4]:
import xgboost as xgb

model = xgb.XGBRegressor()
 
model.fit(x, y)

importances = model.feature_importances_
print(importances)

[0.02177803 0.02388705 0.04590705 0.03498765 0.04350351 0.03919521
 0.03852011 0.04607506 0.04492055 0.04621455 0.0447857  0.04628607
 0.12066203 0.14340933 0.0400168  0.21985121]


In [5]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

x_train_tensor = torch.tensor(x_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)

x_test_tensor = torch.tensor(x_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

train_dataset = TensorDataset(x_train_tensor, y_train_tensor)
train_loader = DataLoader(dataset=train_dataset, batch_size=256, shuffle=True)

test_dataset = TensorDataset(x_test_tensor, y_test_tensor)
test_loader = DataLoader(dataset=test_dataset, batch_size=256, shuffle=True)

In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [7]:
input_size = x.shape[1]
hidden_size = 128
output_size = y.shape[1]
print(input_size)
print(output_size)

# model = MLP(input_size, hidden_size, output_size)
model = MLP_v2(input_size, hidden_size, output_size).to(device)

16
2


In [8]:
#model test
mean_abs_error = model_test(model, test_loader, device)
print(f'Mean Absolute Error: {mean_abs_error:.4f}')

Mean Absolute Error: 19.5908


In [9]:
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

In [10]:
num_epochs = 700
mean_absolute_error_list = []

for epoch in range(num_epochs):
    model.train()
    for inputs, targets in train_loader:
        inputs, targets = inputs.to(device), targets.to(device)
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    if (epoch) % 50 == 0:
        mean_abs_error = model_test(model, test_loader, device)
        mean_absolute_error_list.append(mean_abs_error)
        print(f'Epoch [{epoch}/{num_epochs}], Loss: {loss.item():.4f}, Mean Absolute Error: {mean_abs_error:.4f}')
        # print(f'Epoch [{epoch}/{num_epochs}], Loss: {loss.item():.4f}')


Epoch [0/700], Loss: 331.4535, Mean Absolute Error: 18.2375
Epoch [50/700], Loss: 272.3319, Mean Absolute Error: 15.6251
Epoch [100/700], Loss: 120.5768, Mean Absolute Error: 9.8207
Epoch [150/700], Loss: 55.3271, Mean Absolute Error: 6.2663
Epoch [200/700], Loss: 30.4602, Mean Absolute Error: 5.6956
Epoch [250/700], Loss: 34.7455, Mean Absolute Error: 5.7463
Epoch [300/700], Loss: 25.7126, Mean Absolute Error: 5.8220
Epoch [350/700], Loss: 34.3889, Mean Absolute Error: 5.8970
Epoch [400/700], Loss: 26.5697, Mean Absolute Error: 5.9686
Epoch [450/700], Loss: 26.3554, Mean Absolute Error: 5.8756
Epoch [500/700], Loss: 22.5659, Mean Absolute Error: 5.9532
Epoch [550/700], Loss: 39.2929, Mean Absolute Error: 6.0652
Epoch [600/700], Loss: 23.7221, Mean Absolute Error: 5.9590
Epoch [650/700], Loss: 27.5492, Mean Absolute Error: 6.1413
